# Chapter 4 - Content Based Recommenders

In this chapter, we are going to build two types of content-based recommender:
**
Plot description-based recommender**: This mode 
compares the descriptions and taglines of different movies, a d
provides recommendations that have the most similar p ot
descripti

**ns.
Metadata-based recomme**nder: This model takes a ho t offeatures, such as genres, keywords, cast, and crew, intoconsideration and pro ides recommendations that are the mostsimilar with respect to the aforementioned features.

In [1]:
# Initial Libraries
import pandas as pd
import numpy as np

In [2]:
# Read data
file = 'data/metadata_clean.csv'

df = pd.read_csv(file, low_memory=False)

df.head()

,title,genres,runtime,vote_average,vote_count,year
0,Toy Story,"['animation', 'comedy', 'family']",81.0,7.7,5415.0,1995
1,Jumanji,"['adventure', 'fantasy', 'family']",104.0,6.9,2413.0,1995
2,Grumpier Old Men,"['romance', 'comedy']",101.0,6.5,92.0,1995
3,Waiting to Exhale,"['comedy', 'drama', 'romance']",127.0,6.1,34.0,1995
4,Father of the Bride Part II,['comedy'],106.0,5.7,173.0,1995


In [3]:
#Import the original file
orig_df = pd.read_csv('data/movies_metadata.csv', low_memory=False)

#Add the useful features into the cleaned dataframe
df['overview'], df['id'] = orig_df['overview'], orig_df['id']

df.head()

,title,genres,runtime,vote_average,vote_count,year,overview,id
0,Toy Story,"['animation', 'comedy', 'family']",81.0,7.7,5415.0,1995,"Led by Woody, Andy's toys live happily in his ...",862
1,Jumanji,"['adventure', 'fantasy', 'family']",104.0,6.9,2413.0,1995,When siblings Judy and Peter discover an encha...,8844
2,Grumpier Old Men,"['romance', 'comedy']",101.0,6.5,92.0,1995,A family wedding reignites the ancient feud be...,15602
3,Waiting to Exhale,"['comedy', 'drama', 'romance']",127.0,6.1,34.0,1995,"Cheated on, mistreated and stepped on, the wom...",31357
4,Father of the Bride Part II,['comedy'],106.0,5.7,173.0,1995,Just when George Banks has recovered from his ...,11862


### CountVectorizer

CountVectorizer is the simplest type of vectorizer and is best explained with the help of an example. Imagine that we have three documents, A, B, and C,
which are as follows:

    A: The sun is a star.
    B: My love is like a red, red rose
    C: Mary had a little lamb

We now have to convert these documents into their vector forms using CountVectorizer. The first step is to compute the size of the vocabulary. The
vocabulary is the number of unique words present across all documents. Therefore, the vocabulary for this set of three documents is as follows: the,
sun, is, a, star, my, love, like, red, rose, mary, had, little, lamb. Consequently, the size of the vocabulary is 14.

It is common practice to not include extremely common words such as a, the, is, had, my, and so on (also known as stop words) in the vocabulary.
Therefore, eliminating the stop words, our vocabulary, V, is as follows:

V: like, little, lamb, love, mary, red, rose, sun, star

The size of our vocabulary is now nine. Therefore, our documents will be represented as nine-dimensional vectors, and each dimension here will
represent the number of times a particular word occurs in a document. In other words, the first dimension will represent the number of times like occurs, the second will represent the number of times little occurs, and so on. Therefore, using the CountVectorizer approach, A, B, and C will now be
represented as follows:

    A: (0, 0, 0, 0, 0, 0, 0, 1, 1)
    B: (1, 0, 0, 1, 0, 2, 1, 0, 0)
    C: (0, 1, 1, 0, 1, 0, 0, 0, 0)

<div style="text-align:center;">
    <img src='images/tfidf.jpg'>
</div>

The weight $w_{i,j}$ takes values between 0 and 1.

In [4]:
#Import TfIdfVectorizer from the scikit-learn library
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stopwords
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
df['overview'] = df['overview'].fillna('')

#Construct the required TF-IDF matrix by applying the fit_transform method on the overview feature
tfidf_matrix = tfidf.fit_transform(df['overview'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(45032, 75357)

<div style="text-align:center;">
    <img src='images/css.jpg' width='600'>
</div>

In [5]:
# Import linear_kernel to compute the dot product
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [6]:
#Construct a reverse mapping of indices and movie titles, and drop duplicate titles, if any
indices = pd.Series(df.index, index=df['title']).drop_duplicates()

indices

title
Toy Story                          0
Jumanji                            1
Grumpier Old Men                   2
Waiting to Exhale                  3
Father of the Bride Part II        4
                               ...  
Caged Heat 3000                45027
Robin Hood                     45028
Century of Birthing            45029
Betrayal                       45030
Queerama                       45031
Length: 45032, dtype: int64

### Plot description based recommender

Our plot description-based recommender will take in a movie title as an argument and output a list of movies that are most similar based on their
plots. These are the steps we are going to perform in building this model:

1. Obtain the data required to build the model
2. Create TF-IDF vectors for the plot description (or overview) of every movie
3. Compute the pairwise cosine similarity score of every movie
4. Write the recommender function that takes in a movie title as an argument and outputs movies most similar to it based on the plot

### Recommender Function

We will perform the following steps in building the recommender function:

1. Declare the title of the movie as an argument.
2. Obtain the index of the movie from the indices reverse mapping.
3. Get the list of cosine similarity scores for that particular movie with all movies using cosine_sim. Convert this into a list of tuples where the first element is the position and the second is the similarity score.
4. Sort this list of tuples on the basis of the cosine similarity scores.
5. Get the top 10 elements of this list. Ignore the first element as it refers to the similarity score with itself (the movie most similar to a particular movie is obviously the movie itself).
6. Return the titles corresponding to the indices of the top 10 elements, excluding the first:

In [7]:
# Function that takes in movie title as input and gives recommendations 

def content_recommender(title, cosine_sim=cosine_sim, df=df, indices=indices):
    # Obtain the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    # And convert it into a list of tuples as described above
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the cosine similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies. Ignore the first movie.
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return df['title'].iloc[movie_indices]

### Get Recommendations for a Movie

In [8]:
#Get recommendations for The Lion King
content_recommender('The Lion King')

34682               Manson Family Vacation
9353                    Tipping the Velvet
9115                              Thursday
42829                              Big Jet
25654    Cheech & Chong Get Out of My Room
17041                     Bird of Paradise
27933                      Little Monsters
6094                  Pauline at the Beach
37409                   The Driftless Area
3203                        Beyond the Mat
Name: title, dtype: object

# Metadata Based Recommender

To build this model, we will be using the following metdata:
* The genre of the movie.
* The director of the movie. This person is part of the crew.
* The movie's three major stars. They are part of the cast.
* Sub-genres or keywords.

In [9]:
# Load the keywords and credits files
cred_df = pd.read_csv('data/credits.csv')
key_df = pd.read_csv('data/keywords.csv')

In [10]:
#Print the head of the credit dataframe
cred_df.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [11]:
cred_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [12]:
#Print the head of the keywords dataframe
key_df.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [13]:
key_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46419 entries, 0 to 46418
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        46419 non-null  int64 
 1   keywords  46419 non-null  object
dtypes: int64(1), object(1)
memory usage: 725.4+ KB


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45032 entries, 0 to 45031
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         45032 non-null  object 
 1   genres        45032 non-null  object 
 2   runtime       44786 non-null  float64
 3   vote_average  45032 non-null  float64
 4   vote_count    45032 non-null  float64
 5   year          45032 non-null  int64  
 6   overview      45032 non-null  object 
 7   id            45032 non-null  object 
dtypes: float64(3), int64(1), object(4)
memory usage: 2.7+ MB


In [15]:
# Function to convert all non-integer IDs to NaN
def clean_ids(x):
    try:
        return int(x)
    except:
        return np.nan


#Clean the ids of df
df['id'] = df['id'].apply(clean_ids)

#Filter all rows that have a null ID
df = df[df['id'].notnull()]

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45029 entries, 0 to 45031
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         45029 non-null  object 
 1   genres        45029 non-null  object 
 2   runtime       44783 non-null  float64
 3   vote_average  45029 non-null  float64
 4   vote_count    45029 non-null  float64
 5   year          45029 non-null  int64  
 6   overview      45029 non-null  object 
 7   id            45029 non-null  float64
dtypes: float64(4), int64(1), object(3)
memory usage: 3.1+ MB


In [16]:
# Merge keywords and credits into your main metadata dataframe
df = df.merge(cred_df, on='id')
df = df.merge(key_df, on='id')

#Display the head of df
df.head()

,title,genres,runtime,vote_average,vote_count,year,overview,id,cast,crew,keywords
0,Toy Story,"['animation', 'comedy', 'family']",81.0,7.7,5415.0,1995,"Led by Woody, Andy's toys live happily in his ...",862.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,Jumanji,"['adventure', 'fantasy', 'family']",104.0,6.9,2413.0,1995,When siblings Judy and Peter discover an encha...,8844.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,Grumpier Old Men,"['romance', 'comedy']",101.0,6.5,92.0,1995,A family wedding reignites the ancient feud be...,15602.0,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,Waiting to Exhale,"['comedy', 'drama', 'romance']",127.0,6.1,34.0,1995,"Cheated on, mistreated and stepped on, the wom...",31357.0,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...","[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,Father of the Bride Part II,['comedy'],106.0,5.7,173.0,1995,Just when George Banks has recovered from his ...,11862.0,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


### Wrangling keywords, cast, and crew

Now that we have all the desired features in a single DataFrame, let's convert them into a form that is more usable. More specifically, these are the transformations we will be looking to perform:

* Convert keywords into a list of strings where each string is a keyword (similar to genres). We will include only the top three keywords. Therefore, this list can have a maximum of three elements.

* Convert cast into a list of strings where each string is a star. Like keywords, we will only include the top three stars in our cast. 

* Convert crew into director. In other words, we will extract only the director of the movie and ignore all other crew members.

In [17]:
# Convert the stringified objects into the native python objects
from ast import literal_eval

features = ['cast', 'crew', 'keywords', 'genres']

for feature in features:
    df[feature] = df[feature].apply(literal_eval)

In [18]:
#Print the first cast member of the first movie in df
df.iloc[0]['crew'][0]

{'credit_id': '52fe4284c3a36847f8024f49',
 'department': 'Directing',
 'gender': 2,
 'id': 7879,
 'job': 'Director',
 'name': 'John Lasseter',
 'profile_path': '/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg'}

In [19]:
# Extract the director's name. If director is not listed, return NaN
def get_director(x):
    for crew_member in x:
        if crew_member['job'] == 'Director':
            return crew_member['name']
    return np.nan

#Define the new director feature
df['director'] = df['crew'].apply(get_director)

df.head()

,title,genres,runtime,vote_average,vote_count,year,overview,id,cast,crew,keywords,director
0,Toy Story,"[animation, comedy, family]",81.0,7.7,5415.0,1995,"Led by Woody, Andy's toys live happily in his ...",862.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...",John Lasseter
1,Jumanji,"[adventure, fantasy, family]",104.0,6.9,2413.0,1995,When siblings Judy and Peter discover an encha...,8844.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1...",Joe Johnston
2,Grumpier Old Men,"[romance, comedy]",101.0,6.5,92.0,1995,A family wedding reignites the ancient feud be...,15602.0,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392...",Howard Deutch
3,Waiting to Exhale,"[comedy, drama, romance]",127.0,6.1,34.0,1995,"Cheated on, mistreated and stepped on, the wom...",31357.0,"[{'cast_id': 1, 'character': 'Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...","[{'id': 818, 'name': 'based on novel'}, {'id':...",Forest Whitaker
4,Father of the Bride Part II,[comedy],106.0,5.7,173.0,1995,Just when George Banks has recovered from his ...,11862.0,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n...",Charles Shyer


In [20]:
# Returns the list top 3 elements or entire list; whichever is more.
def generate_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        #Check if more than 3 elements exist. If yes, return only first three. If no, return entire list.
        if len(names) > 3:
            names = names[:3]
        return names

    #Return empty list in case of missing/malformed data
    return []

#Apply the generate_list function to cast and keywords
df['cast'] = df['cast'].apply(generate_list)
df['keywords'] = df['keywords'].apply(generate_list)

df.head()

,title,genres,runtime,vote_average,vote_count,year,overview,id,cast,crew,keywords,director
0,Toy Story,"[animation, comedy, family]",81.0,7.7,5415.0,1995,"Led by Woody, Andy's toys live happily in his ...",862.0,"[Tom Hanks, Tim Allen, Don Rickles]","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[jealousy, toy, boy]",John Lasseter
1,Jumanji,"[adventure, fantasy, family]",104.0,6.9,2413.0,1995,When siblings Judy and Peter discover an encha...,8844.0,"[Robin Williams, Jonathan Hyde, Kirsten Dunst]","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[board game, disappearance, based on children'...",Joe Johnston
2,Grumpier Old Men,"[romance, comedy]",101.0,6.5,92.0,1995,A family wedding reignites the ancient feud be...,15602.0,"[Walter Matthau, Jack Lemmon, Ann-Margret]","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[fishing, best friend, duringcreditsstinger]",Howard Deutch
3,Waiting to Exhale,"[comedy, drama, romance]",127.0,6.1,34.0,1995,"Cheated on, mistreated and stepped on, the wom...",31357.0,"[Whitney Houston, Angela Bassett, Loretta Devine]","[{'credit_id': '52fe44779251416c91011acb', 'de...","[based on novel, interracial relationship, sin...",Forest Whitaker
4,Father of the Bride Part II,[comedy],106.0,5.7,173.0,1995,Just when George Banks has recovered from his ...,11862.0,"[Steve Martin, Diane Keaton, Martin Short]","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[baby, midlife crisis, confidence]",Charles Shyer


In [21]:
#Only consider a maximum of 3 genres
df['genres'] = df['genres'].apply(lambda x: x[:3])

In [22]:
# Print the new features of the first 5 movies along with title
df[['title', 'cast', 'director', 'keywords', 'genres']].head()

,title,cast,director,keywords,genres
0,Toy Story,"[Tom Hanks, Tim Allen, Don Rickles]",John Lasseter,"[jealousy, toy, boy]","[animation, comedy, family]"
1,Jumanji,"[Robin Williams, Jonathan Hyde, Kirsten Dunst]",Joe Johnston,"[board game, disappearance, based on children'...","[adventure, fantasy, family]"
2,Grumpier Old Men,"[Walter Matthau, Jack Lemmon, Ann-Margret]",Howard Deutch,"[fishing, best friend, duringcreditsstinger]","[romance, comedy]"
3,Waiting to Exhale,"[Whitney Houston, Angela Bassett, Loretta Devine]",Forest Whitaker,"[based on novel, interracial relationship, sin...","[comedy, drama, romance]"
4,Father of the Bride Part II,"[Steve Martin, Diane Keaton, Martin Short]",Charles Shyer,"[baby, midlife crisis, confidence]",[comedy]


In [23]:
# Function to sanitize data to prevent ambiguity. It removes spaces and converts to lowercase
def sanitize(x):
    if isinstance(x, list):
        #Strip spaces and convert to lowercase
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''


#Apply the generate_list function to cast, keywords, director and genres
for feature in ['cast', 'director', 'genres', 'keywords']:
    df[feature] = df[feature].apply(sanitize)

### Create Soup

In [24]:
#Function that creates a soup out of the desired metadata
def create_soup(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])

# Create the new soup feature
df['soup'] = df.apply(create_soup, axis=1)

df.head()

,title,genres,runtime,vote_average,vote_count,year,overview,id,cast,crew,keywords,director,soup
0,Toy Story,"[animation, comedy, family]",81.0,7.7,5415.0,1995,"Led by Woody, Andy's toys live happily in his ...",862.0,"[tomhanks, timallen, donrickles]","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[jealousy, toy, boy]",johnlasseter,jealousy toy boy tomhanks timallen donrickles ...
1,Jumanji,"[adventure, fantasy, family]",104.0,6.9,2413.0,1995,When siblings Judy and Peter discover an encha...,8844.0,"[robinwilliams, jonathanhyde, kirstendunst]","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[boardgame, disappearance, basedonchildren'sbook]",joejohnston,boardgame disappearance basedonchildren'sbook ...
2,Grumpier Old Men,"[romance, comedy]",101.0,6.5,92.0,1995,A family wedding reignites the ancient feud be...,15602.0,"[waltermatthau, jacklemmon, ann-margret]","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[fishing, bestfriend, duringcreditsstinger]",howarddeutch,fishing bestfriend duringcreditsstinger walter...
3,Waiting to Exhale,"[comedy, drama, romance]",127.0,6.1,34.0,1995,"Cheated on, mistreated and stepped on, the wom...",31357.0,"[whitneyhouston, angelabassett, lorettadevine]","[{'credit_id': '52fe44779251416c91011acb', 'de...","[basedonnovel, interracialrelationship, single...",forestwhitaker,basedonnovel interracialrelationship singlemot...
4,Father of the Bride Part II,[comedy],106.0,5.7,173.0,1995,Just when George Banks has recovered from his ...,11862.0,"[stevemartin, dianekeaton, martinshort]","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[baby, midlifecrisis, confidence]",charlesshyer,baby midlifecrisis confidence stevemartin dian...


In [25]:
#Display the soup of the first movie
df.iloc[0]['soup']

'jealousy toy boy tomhanks timallen donrickles johnlasseter animation comedy family'

### TF-IDF Vectorizer

In [26]:
# Import CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

#Define a new CountVectorizer object and create vectors for the soup
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(df['soup'])

### Compute the Cosine Similarity

In [27]:
#Import cosine_similarity function
from sklearn.metrics.pairwise import cosine_similarity

#Compute the cosine similarity score (equivalent to dot product for tf-idf vectors)
cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

### Construct Reverse Mapping

In [28]:
# Reset index of your df and construct reverse mapping again
df = df.reset_index()
indices2 = pd.Series(df.index, index=df['title'])

### Get Recommendations for a Movie

In [29]:
content_recommender('The Lion King', cosine_sim2, df, indices2)

29722                            Barbie of Swan Lake
31433                            Flight of the Doves
43525                                     Fair Haven
15489                             The Halloween Tree
41562                          My Life as a Zucchini
5674     The Looney, Looney, Looney Bugs Bunny Movie
6678                                      Kart Racer
12985                                  The Longshots
16002                                 The Eighth Day
17658                                Alpha and Omega
Name: title, dtype: object